<a href="https://colab.research.google.com/github/luasm17/LLM_as_a_judge/blob/main/qwen3_4B_few_shot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Instalar dependencias + Imports
!pip install -q -U "transformers>=4.51.0" accelerate safetensors

import os
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 101.7 MB/s eta 0:00:00


In [2]:
# BLOQUE: Prompt

USER_PROMPT = r'''Es un LLM-as-a-judge que vai avaliar un modelo de corrección gramatical (GEC) en galego. En concreto, a túa tarefa é a de avaliar a saída dun corrector gramatical e decidir se o modelo GEC corrixiu correctamente o erro de concordancia de número da frase inicial (é dicir, singular/plural entre determinante, substantivo, adxectivo ou verbo). Lembra que debes cinguirte exclusivamente á corrección ou non da concordancia de número.



Recibirás sempre:

input_corrector: a frase orixinal que se lle pasou ao corrector

output_corrector: a frase devolta polo corrector



Tes que devolver a túa avaliación como LLM-as-a-judge seguindo exactamente este formato; debes respostar exclusivamente a estes catro puntos variables, sen engadir texto adicional:

input_corrector: "<oración de entrada do corrector>"

output_corrector: "<oración xa avaliada polo corrector>"

tag: <0 ou 1>

explanation: "<explicación breve e precisa en galego do motivo polo que se escolleu a etiqueta 0 ou a etiqueta 1>"



Criterios que debes seguir:

tag = 0: a saída do corrector é correcta con respecto á concordancia de número (é dicir, non hai erro de concordancia de número no output_corrector). Nota sobre a tag = 0 (dúas situacións posibles):

Caso 0.A: input_corrector = output_corrector. O input_corrector xa era correcto respecto á concordancia de número e o modelo GEC non fixo cambios adicionais porque non había nada que corrixir. A explicación debe indicar que non había erro de concordancia de número no input.

Caso 0.B: input_corrector ≠ output_corrector. O input_corrector tiña un erro de concordancia de número e o modelo GEC corrixiuno. A explicación debe indicar que erro(s) concordancia se arranxou/arranxaron.

En ambos casos, a etiqueta é 0, pero a explicación debe reflectir se houbo corrección do input_corrector (caso 0.B) ou non por ser xa correcto (caso 0.A).



tag = 1: a saída do corrector non é correcta con respecto á concordancia de número (é dicir, segue habendo erro de concordancia de número no output_corrector). Nota sobre a tag = 1 (dúas situacións posibles):

Caso 1.A: input_corrector = output_corrector. O input_corrector era incorrecto respecto á concordancia de número e o modelo GEC non correxiu o erro presente. A explicación debe indicar que o modelo GEC non corrixiu o erro de concordancia de número presente no input_corrector.

Caso 1.B: input_corrector ≠ output_corrector. O input_corrector non tiña ningún erro de concordancia de número, pero o modelo GEC introduciuno. A explicación debe indicar que o corrector introduciu no output_corrector un erro que non estaba presente no input_corrector.

En ambos casos, a etiqueta é 1, pero a explicación debe reflectir se houbo modificación do input_corrector (caso 1.B) ou non (caso 1.A).



Restricións que debes respectar:

Non debes, baixo ningún concepto, corrixir ou modificar de ningunha forma o input nin o output do corrector.

Tes que limitarte exclusivamente a decidir se o erro de concordancia de número foi corrixido polo modelo GEC ou non.

É imprescindible que te fixes só na concordancia de número, non debes avaliar, baixo ningún concepto, outros tipos de erros.

Non uses nunca o español, o portugués nin nigunha lingua que non sexa o galego nas túas respostas.



Aquí tes uns exemplos de inputs que poderías atopar e dos output que deberías devolver, respectivamente:



Exemplo 1 (caso 0.A): tag = 0

Exemplo de entrada (o que che vai chegar):

input_corrector: "Acaba de saír o sol despois de moita choiva e os nenos corren cara o parque."

output_corrector: "Acaba de saír o sol despois de moita choiva e os nenos corren cara o parque."

Exemplo de saída (o que ti tes que devolver):

input_corrector: "Acaba de saír o sol despois de moita choiva e os nenos corren cara o parque."

output_corrector: "Acaba de saír o sol despois de moita choiva e os nenos corren cara o parque."

tag: 0

explanation: "O output_corrector é adecuado con respecto á concordancia de número xa que non se modificou o input_corrector, o cal non contiña ningún erro de concordancia que corrixir."



Exemplo 2 (caso 0.B): tag = 0

Exemplo de entrada (o que che vai chegar):

input_corrector: "A decisións tomadas polo comité foron comunicadas aos responsables das distintas áreas."

output_corrector: "As decisións tomadas polo comité foron comunicadas aos responsables das distintas áreas."



Exemplo de saída (o que ti tes que devolver):

input_corrector: "A decisións tomadas polo comité foron comunicadas aos responsables das distintas áreas."

output_corrector: "As decisións tomadas polo comité foron comunicadas aos responsables das distintas áreas."

tag: 0

explanation: "O output_corrector é adecuado con respecto á concordancia de número porque solucionou o erro do input_corrector entre “A"e “decisións”."



Exemplo 3 (caso 1.A): tag = 1

Exemplo de entrada (o que che vai chegar):

input_corrector: "Pásame ese libro que están enriba da mesa, por favor."

output_corrector: "Pásame ese libro que están enriba da mesa, por favor."



Exemplo de saída (o que ti tes que devolver):

input_corrector: "Pásame ese libro que están enriba da mesa, por favor."

output_corrector: "Pásame ese libro que están enriba da mesa, por favor."

tag: 1

explanation: "O output_corrector non é adecuado con respecto á concordancia de número porque non se modificou o input_corrector, o cal contén un erro de concordancia entre “están"e “libro”."



Exemplo 4 (caso 1.B): tag = 1

Exemplo de entrada (o que che vai chegar):

input_corrector: "Todo o mundo chegou a tempo á xuntanza aquel día."

output_corrector: "Todo o mundo chegaron a tempo á xuntanza aquel día."



Exemplo de saída (o que ti tes que devolver):

input_corrector: "Todo o mundo chegou a tempo á xuntanza aquel día."

output_corrector: "Todo o mundo chegaron a tempo á xuntanza aquel día."

tag: 1

explanation: "O output_corrector non é adecuado con respecto á concordancia de número porque o corrector engadiu un erro que non existía no input_corrector."





Agora, avalía os seguintes casos:
'''

def build_prompt(input_text: str, output_text: str) -> str:
    return (
        USER_PROMPT
        + "\n\n"
        + f'input_corrector: "{input_text}"\n'
        + f'output_corrector: "{output_text}"\n'
    )


In [3]:
# Cargamos el archivo CSV con las frases corregidas.
# Cada fila representa un par input-output del corrector.

file_path = "/content/frases_corrector.csv"
df = pd.read_csv(file_path, encoding='UTF-8', delimiter=';')

In [4]:
# Token HF
HF_TOKEN = os.environ.get("HF_TOKEN", None)

In [5]:
# Cargar modelo e tokenizer (Qwen3-4B-Instruct-2507)
# LLM-as-a-Judge binario para concordancia de número en galego

MODEL_ID = "Qwen/Qwen3-4B-Instruct-2507"

print("🔄 Cargando tokenizer...")
if HF_TOKEN:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, token=HF_TOKEN)
else:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

print("🔄 Cargando modelo (pode tardar uns minutos)...")
if HF_TOKEN:
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        token=HF_TOKEN,
        device_map="auto",
        torch_dtype="auto"
    )
else:
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        device_map="auto",
        torch_dtype="auto"
    )

model.eval()
print("✅ Modelo cargado correctamente")

🔄 Cargando tokenizer...


config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/398 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

✅ Modelo cargado correctamente


In [6]:
# BLOQUE: Post-procesado para imprimir el output no formato pedido ---
import re

TAG_RE = re.compile(r"^\s*tag\s*:\s*([01])\s*$", re.IGNORECASE | re.MULTILINE)
EXP_RE = re.compile(r"^\s*explanation\s*:\s*(.*)$", re.IGNORECASE | re.MULTILINE)

def force_strict_4lines(input_text: str, output_text: str, raw: str) -> str:
    tag_m = TAG_RE.search(raw)
    tag = tag_m.group(1) if tag_m else None
    exp_m = EXP_RE.search(raw)
    explanation = exp_m.group(1).strip() if exp_m else None
    if explanation is None:
        explanation = raw.strip()
    explanation = " ".join(explanation.split())
    if tag is None:
        m = re.search(r"\btag\s*:\s*([01])\b", raw, re.IGNORECASE)
        tag = m.group(1) if m else "0"
    return (
        f'input_corrector: "{input_text}"\n'
        f'output_corrector: "{output_text}"\n'
        f'tag: {tag}\n'
        f'explanation: "{explanation}"'
    )


In [7]:
# Inferencia e impresión
# Cambios mínimos: 1) decodificamos só os tokens xerados (sen o prompt), 2) imprimimos só 4 liñas.

import pandas as pd
import torch

if 'df' not in globals():
    df = pd.read_csv('frases_corrector.csv', encoding='latin1', delimiter=';')

# Rename the column to remove the BOM character if it exists
if 'ï»¿input_corrector' in df.columns:
    df.rename(columns={'ï»¿input_corrector': 'input_corrector'}, inplace=True)

for _, row in df.iterrows():
    input_corrector = row['input_corrector']
    output_corrector = row['output_corrector']

    prompt = build_prompt(input_corrector, output_corrector)

    # Qwen3: usar chat_template cando exista. Para evitar eco do prompt,
    # decodificamos só os tokens novos.
    if hasattr(tokenizer, 'apply_chat_template') and getattr(tokenizer, 'chat_template', None):
        messages = [
            {'role': 'system', 'content': 'Devolve EXCLUSIVAMENTE o output final co formato pedido no prompt. Non escribas "user" nin repitas o prompt.'},
            {'role': 'user', 'content': prompt},
        ]
        text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        inputs = tokenizer(text, return_tensors='pt')
    else:
        inputs = tokenizer(prompt, return_tensors='pt')

    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    prompt_len = inputs['input_ids'].shape[1]

    with torch.no_grad():
        generated = model.generate(
            **inputs,
            max_new_tokens=256,
            do_sample=False,
        )

    gen_ids = generated[0][prompt_len:]
    raw_out = tokenizer.decode(gen_ids, skip_special_tokens=True).strip()
    final_out = force_strict_4lines(input_corrector, output_corrector, raw_out)
    print(final_out)
    print('-' * 60)


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


input_corrector: "As eleccións para escoller o novo presidente da comunidade non tiveron lugar por mor do mal ambiente no condominio."
output_corrector: "As eleccións para elixir o novo presidente da comunidade non tiveron lugar por mor do mal ambiente no condominio."
tag: 0
explanation: ""O output_corrector é adecuado con respecto á concordancia de número porque solucionou o erro do input_corrector entre 'escoller' e 'elixir', que xa estaba incorrecto no input_corrector.""
------------------------------------------------------------
input_corrector: "O grupo de estudantes do proxecto presentaron os resultados finais onte pola tarde."
output_corrector: "O grupo de estudantes do proxecto presentou os resultados finais onte pola tarde."
tag: 1
explanation: ""O output_corrector non é adecuado con respecto á concordancia de número porque non se modificou o input_corrector, o cal contén un erro de concordancia entre «presentaron» e «grupo», xa que o verbo debe estar no singular por coincidi